# Atmospherically Corrected Earth Engine Time Series 

### Overview

This notebook creates atmospherically corrected time series of satellite imagery using [Google Earth Engine](https://earthengine.google.com/) and the [6S emulator](https://github.com/samsammurphy/6S_emulator). 

### Supported missions

* Sentintel2
* Landsat8
* Landsat7
* Landsat5
* Landsat4

### Output

Average, cloud-*free* pixel values 

### Cloud masking

Uses standard cloud masks, i.e. FMASK for Landsat and ESA-QA60 for Sentinel 2.  There is no guarantee they will find all clouds, a discussion on more advance and/or alternative cloud masking strategies is available [here](https://groups.google.com/forum/#!searchin/google-earth-engine-developers/cloud$20AND$20sentinel2%7Csort:relevance/google-earth-engine-developers/i63DS-Dg8Sg/FWenONUFBwAJ)

#### Initialize

In [1]:
# standard modules
import os
import sys
import ee
import matplotlib
import pandas as pd
%matplotlib inline
ee.Initialize()

# custom modules
base_dir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(base_dir,'atmcorr'))
from kml_reader import read_kml
from ee_requests import request_meanRadiance
from data_extraction import *

from timeSeries import timeSeries
from postProcessing import postProcessing
from plots import *


In [2]:
#debuggin
%load_ext autoreload
%autoreload 2

### User Input

In [27]:
target = 'Poas'

# geometry
geom = read_kml('IAVCEI.kml', target)

# start and end of time series
startDate = '1985-01-01'# YYYY-MM-DD
stopDate  = '2017-01-01'# YYYY-MM-DD

# satellite missions
missions = ['Sentinel2', 'Landsat8', 'Landsat7', 'Landsat5', 'Landsat4']

### Google Earth Engine TimeSeries

We need to extract data from Earth Engine so that we can process it locally. The following code gets the mean average radiance through time from inside the geometry defined above (clouds are masked by default). 

It also grabs the inputs required for atmospheric correction (e.g. aerosol optical thickness, water vapor, etc.).

#### Earth Engine Requests
Let's set up some requests for Earth Engine data.

In [28]:
requests = {}

for mission in missions:
    requests[mission] = request_meanRadiance(geom, startDate, stopDate, mission, True)

#### Execute request

There are two methods for  accessing this data locally.

1. getInfo 
2. export 

The **getInfo()** option is easier for humans, we will try that first. The **export** option can handle larger requests (it is our backup plan).

In [29]:
try:
    data = data_from_getInfo(requests)
except Exception as e:
    task = data_export_task(requests, target)
    data = task_manager(task, target, e)

Requesting data locally
Sentinel2..
Landsat8..
Landsat7..
Landsat5..
Landsat4..


In [30]:
# maybe check for data here and try loading from csv if None throw error (i.e. to pause the nb)

In [31]:
data

,imageID,mission,B1,B10,B11,B12,B2,B3,B4,B5,...,B7,B8,B8A,B9,alt,aot,doy,h2o,o3,solar_z
2015-12-03 16:11:28.402,20151203T161128_20151203T195130_T16PGS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.117000,337.674634,3.680000,0.242000,37.660487
2015-12-03 16:11:28.402,20151203T161128_20151203T195130_T16PHS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.117000,337.674634,3.680000,0.242000,37.212904
2016-01-02 16:11:25.753,20160102T161125_20160102T195442_T16PGS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.072000,2.674604,3.180000,0.245724,40.175480
2016-01-02 16:11:25.753,20160102T161125_20160102T195442_T16PHS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.072000,2.674604,3.180000,0.245724,39.680492
2016-02-01 16:07:56.738,20160201T160756_20160201T211039_T16PGS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.097000,32.672184,2.800000,0.239000,37.234496
2016-02-01 16:07:56.738,20160201T160756_20160201T211039_T16PHS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.097000,32.672184,2.800000,0.239000,36.632694
2016-03-02 16:06:54.135,20160302T160654_20160302T204848_T16PGS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.155000,62.671460,3.260000,0.230000,30.303746
2016-03-02 16:06:54.135,20160302T160654_20160302T204848_T16PHS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.155000,62.671460,3.260000,0.230000,29.564239
2016-04-01 16:10:17.963,20160401T161017_20160401T230454_T16PGS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.145000,92.673819,4.160000,0.243000,23.392510
2016-04-01 16:10:17.963,20160401T161017_20160401T230454_T16PHS,Sentinel2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.358,0.145000,92.673819,4.160000,0.243000,22.522877


### Atmospherically correct the data.

In [26]:
from atmospheric_correction import run_atmcorr
data = run_atmcorr(data)

In [ ]:
allTimeSeries = timeSeries(target, geom, startDate, stopDate, missions)

### Data post-processing
Resample into daily intervals using liner interpolation and calculate hue-saturation-value from RGB.

In [ ]:
from postProcessing import postProcessing
data = postProcessing(allTimeSeries)

### bringing it all together...
make a pretty graph to help us do some science.

In [ ]:
from plots import *
t1 = pd.datetime.strptime('2005-01-01','%Y-%m-%d')
t2  = pd.datetime.strptime('2017-01-01','%Y-%m-%d')

In [ ]:
fieldData = load_fieldData(target, startDate, stopDate)

In [ ]:
from plots import *
save = True
# save = False
plot_history(target, fieldData, t1, t2, save=save)

In [ ]:
plot_acid(target, fieldData, t1, t2, save=save)

In [ ]:
plot_other(target, fieldData, t1, t2, save=save)

In [ ]:
plot_hueSticks(target, data, t1, t2, save=save)

In [ ]:
plot_color(target, data,'sat', t1, t2, fieldData, save=save)

In [ ]:
plot_color(target, data, 'val', t1, t2, fieldData, save=save)